In [6]:
import base64
import numpy as np
import io
import tensorflow as tf
from PIL import Image
import keras
from keras import backend as K
from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from flask import request
from flask import jsonify
from flask import Flask
from io import BytesIO
from PIL import Image
import cv2
app = Flask(__name__)
def get_model():
    #global model
    #model=load_model('dentalalpha.h5')
    model = keras.models.load_model('dentalalpha.h5')
    #model._make_predict_function()
    #graph = tf.get_default_graph()
    print(" * Model loaded!")

def preprocess_image(image,target_size):
    if image.mode!="RGB":
        image=image.convert("RGB")
    
    image = image.resize(target_size)
    image=img_to_array(image)
    image = np.expand_dims(image,axis=0)
    return image
print(" *Loading Keras model....")
get_model()

@app.route("/predict",methods=["POST"])
def predict():
   
    message=request.get_json(force=True)
    encoded=message['image']
    #image_data=re.sub('^data:image/.+base64,','',encoded).decode('base64')
    #image=Image.open(StringIO.StringIO(image_data))
    decoded=base64.b64decode(encoded)
    image=Image.open(io.BytesIO(decoded))
 
    processed_image=preprocess_image(image,target_size=(224,224))
    #global graph
    #with graph.as_default():
    prediction=model.predict(processed_image).tolist()
    
    response={
        'prediction':
        {
            'dog':prediction[0][0],
            'cat':1-prediction[0][0]
            
        }
    }
    print(prediction[0][0])
    return jsonify(response)
app.run(host='0.0.0.0', port=50000)

 *Loading Keras model....
 * Model loaded!
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:50000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Jun/2019 22:47:00] "POST /predict HTTP/1.1" 200 -


0.7671751379966736


127.0.0.1 - - [07/Jun/2019 22:47:14] "POST /predict HTTP/1.1" 200 -


0.9998482465744019
